In [ ]:
using ModelingToolkit, DifferentialEquations, Plots

In [ ]:
# Define our state variables: state(t) = initial condition
@variables t s(t)=1000 i(t)=10 r(t)=0
@parameters β=0.3 γ=1/4 N=1000

D = Differential(t)

model = [   D(s) ~ - (β * i/ N) * s,
            D(i) ~ (β * i/ N) * s - γ * i,
            D(r) ~ γ * i]

@named ode_model = ODESystem(model, t)

# Convert from a symbolic to a numerical problem to simulate
tspan = (0.0, 120.0)
prob  = ODEProblem(ode_model, [], tspan)

############-############ PLOTS ############-############

# Solve the ODE
sol = solve(prob)

# Plot the solution
p1 = plot(sol, idxs=s, title = "Susceptible")
p2 = plot(sol, idxs=i, title = "Infectious")
p3 = plot(sol, idxs=r, title = "Recovered")

plot(p1, p2, p3, layout = (3, 1))

############-############ - ############-############



## sir with functions

In [ ]:

function sir_model(dx, x, θ, t)
    s, i, r = x
    β, γ, N = θ

    λ = β * i / N

    dx[1] = - λ * s
    dx[2] = λ * s - γ * i
    dx[3] = γ * i
end

In [ ]:
N  = 1e6
i0 = 2/100

# IV
x0 = [N*(1-i0), N*i0, 0]

# Simulation interval
tspan = (0.0, 120.0)

R0 = 3.0
γ  = 1/7

# θ = [α, β, N]
p   = [R0*γ, γ, N]

sir_ode = ODEProblem(sir_model, x0, tspan, p)
ysim    = solve(sir_ode, saveat = 1)
data    = Array(ysim)


In [ ]:
# Plot the solution
p1 = plot(data[1,:], title = "Susceptible")
p2 = plot(data[2,:], title = "Infectious")
p3 = plot(data[3,:], title = "Recovered")

plot(p1, p2, p3, layout = (3, 1))


# Stochastic simulation

In [ ]:
using Random, Distributions, RandomNumbers, Statistics

function plot_quantile(t, y, q, color="black", label="95% CI", legend=:topright)

    up_q  = quantile.(eachrow(y), q/2)
    low_q = quantile.(eachrow(y), 1-q/2)

    p = plot(t, low_q, fillrange = up_q, fillalpha = 0.5, c = color, label = "95% CI", legend = :topright)

    return p
end


In [ ]:
function binomial_transition(x, τ, δτ=1)
    p = 1 .- exp.(-τ .* δt)
    return rand.(Binomial.(x, p))
end

function sir_stochastic(x, θ, t)
    β, γ, N, δt = θ

    s = x[1, :]
    i = x[2, :]
    r = x[3, :]

    λ = β .* i ./ N

    s2i = binomial_transition(s, λ, δt)
    i2r = binomial_transition(i, γ, δt)

    s .= s .- s2i
    i .= i .+ s2i .- i2r
    r .= r .+ i2r

    return transpose(cat(s, i, r, dims=2))
end

R0 = 2.0
γ  = 1/5
N  = 1e6
δt = 1.0

θ  = [R0*γ, γ, N, δt]

T = 5
n = 3
m = 700

i0 = 10/100
x0 = [N*(1-i0), N*i0, 0] * ones(Float64, 1, m);

x         = Array{Float64}(undef, T, n, m)
x[1,:,:] .= x0;

for t in 1:T-1
    x[t+1,:,:] = sir_stochastic(x[t,:,:], θ, t)
end



In [ ]:

q     = 0.975

p1 = plot_quantile(Array(1:1:T), x[:, 1,:], 0.95)
p2 = plot_quantile(Array(1:1:T), x[:, 2,:], 0.95)
p3 = plot_quantile(Array(1:1:T), x[:, 3,:], 0.95)

pp1 = plot(Array(1:1:T), x[:,1,:])

plot(p1, p2, p3, layout = (4, 1))


In [ ]:
ones(1,30)

In [ ]:

function filter(x, y, g, hσ)
    """ Ensemble Adjustment Kalman Filter
    """
    n, m = size(x) # x \in R^{n\times m}. n: state space size, m: number of particles

    μ  = mean(y, dims=2)
    σ  = var(y, dims=2)


    mu_prior  = np.mean(y, -1, keepdims=True)
    var_prior = np.var(y, -1, keepdims=True)

    z   = g(x)  # simulated observations.
    oev = hσ(y) # measurement error.

    for ni in 1:1:n
        xi       = x[ni, :]
        dx       = cov(xi, y)
        x[ni, :] = xi + dx
    end

end

function if2(f, g, h, f0, θo, gθ, δθ, Nif)
    """ Iterated filtering with perturbed Bayes map
    """
    x0 = f0(θo)
    for nif in 1:1:Nif

        if nif == 1
            θ_hat = θo()
        else
            θ_hat = gθ(θ_hat)
        end

        for t in 1:δt:T
            x[t+1,:,:] = f(x[t,:,:], θ, t, δt)
            if t in t_infer

                x, θ = filter(x, y, f, g)
                θ    = δθ(θ)
            end
        end
    end
    return θ_hat
end
